# Homework of Ch4. Convolutional Neural Network
----
This is the homework of TU-ETP-AD1062 Machine Learning Fundamentals.

For more information, please refer to:
https://sites.google.com/view/tu-ad1062-mlfundamentals/

In [ ]:
from keras.datasets import fashion_mnist
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adadelta
from keras.utils import to_categorical

import numpy as np
import pickle
import sklearn.model_selection
import csv

import matplotlib.pyplot as plt
import matplotlib.image as mpimg


## 1. Load and Pre-processing Fashion-MNIST
----
The code here demonstrate how to load the built-in Fashion-MNIST from Keras. Here also includes some pre-processing steps:
1. Load Fashion MNIST data
2. Compute the all available classes
3. Normalize to range `0.0-1.0` by divided with 255
4. Reshape to `(28, 28, 1)`
5. Convert label to categorical array
6. Plot the first 10 images

> **Your task**:  
> Complete step 3 and 5 mentioned above

In [ ]:
# Step 1. Load Fashion MNIST data
(X_train, y_train), (_, _) = fashion_mnist.load_data()

f_test = open('fashion_mnist_test_tu-etp-ad1062-hw4.pickle', 'rb')
X_test = pickle.load(f_test)
f_test.close()

# Step 2. Compute the all available classes
classes = np.unique(y_train)
num_classes = len(classes)

# Step 3. Now it's your turn!: Normalize from 0-255 to 0.0-1.0
# ----------------------------------------------------------------
# X_train = ...
# X_test = ...

# ----------------------------------------------------------------
# Step 3. Checkpoint
assert (np.min(X_train) >= 0.0 and np.max(X_train) <= 1.0), "Your `X_train` range is NOT correct, check should be 0.0 - 1.0"
assert (np.min(X_test) >= 0.0 and np.max(X_test) <= 1.0), "Your `X_test` range is NOT correct, check should be 0.0 - 1.0"

# Step 4. Reshape to (28, 28, 1)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Step 5. Now it's your turn!: Convert label to categorical array
# ----------------------------------------------------------------
# y_train = ...

# ----------------------------------------------------------------
# Step 5. Checkpoint
assert y_train.shape == (60000,10), "Your `y_train` dimension is NOT correct, check and modify the above codeline!"

# Step 6. Plot the first 10 images
plt.figure(figsize=(12, 12))
for i in range(0,10):
    plt.subplot(1, 10, i+1)
    plt.imshow(X_train[i,:,:,0], cmap='gray')

## 2. Construct ConvNet
----
The code shown below constructs a ConvNet with following structure:
1. Convolutional Layer:
    * Kernel size `3 x 3`
    * Number of filters `8`
    * No padding
2. Max Pooling Layer:
    * Pool size `2 x 2`
3. Drop-out Layer:
    * Drop-out rate `0.25`
4. Flatten layer
5. Fully-connected layer with soft-max activation

> **Your task**:  
> Adjust the hierachy of your ConvNet. Check Keras manual for more details. For examples:
> - Convolution Layer: https://keras.io/layers/convolutional/#conv2d
> - Dense Layer (Fully-connected layer): https://keras.io/layers/core/#dense
> - Max Pooling Layer: https://keras.io/layers/pooling/#maxpooling2d

In [ ]:
def create_convNet():
    model = Sequential()
    
    # Now it's your turn!: Adjust your ConvNet
    # ----------------------------------------------------------------
    model.add(Conv2D(8, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(28,28,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adadelta(), metrics=['accuracy'])
    
    return model

## 3. Cross-validation
----
The code shown below conduct 5-fold cross-validation on training set based on the model adjusted above (i.e., `create_convNet`)

> **Your task**:  
> - Keep adjusting `create_convNet()` and execute the following block
> - Make sure that you are happy with the 5-fold cross-validation result!

In [ ]:
neural_network = KerasClassifier(build_fn=create_convNet, epochs=5, batch_size=256, verbose=1)

cv = 5
scores = sklearn.model_selection.cross_val_score(neural_network, X_train, y_train, cv=5)

print("%d-fold Cross Validation Result" % cv)
print(scores)

4. Predict the testing set
----
The code shown below helps you read the testing data, predict with your `create_convNet()`, then output as CSV files

> **Your task:**
> 1. Download testing set from Kaggle website:  https://www.kaggle.com/t/e24b3628c69542239c3d24d465898d03
> 2. Put the testing set downloaded to the same location as this `*.ipynb` file
> 3. Execute the following block to:
>    - Train with `X_train` and `y_train`
>    - Predict with `X_kaggle`
> 4. Upload your evaluation result to Kaggle (NOTICE: 5 submissions per-day!)
> 5. Check your public scoreboard!
> 6. Submit your homework 4 (Google form), make sure to **Fill in your Trend Micro PSID team name!!**

In [ ]:
# Train and predict the test data
neural_network.fit(X_train, y_train)
y_test_predict = neural_network.predict(X_test)

# Output as CSV
id_array = range(0,10000)
y_test_predict.astype('int8')
submission = np.stack((id_array, y_test_predict),axis=1)
np.savetxt("submission.csv", submission, fmt="%d", delimiter=',', header='id,answer', comments='')

print("CONGRATULATIONS! YOU'VE ALREADY DONE! PLEASE SUBMIT YOUR submission.csv to https://www.kaggle.com/t/e24b3628c69542239c3d24d465898d03")